# Implementing a GNN 
*without PyTorch Geometric

In [ ]:
import wntr
import pandas as pd
import numpy as np
import pickle
import networkx as nx

import math
from scipy.sparse import identity

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
# Create a water network model
inp_file = 'Working files/Mod_AnyT_0.inp'
wn_current_WDS = wntr.network.WaterNetworkModel(inp_file)

# Graph the network
wntr.graphics.plot_network(wn_current_WDS, title=wn_current_WDS.name)


In [273]:
S_matrices = []
res_index = []

In [ ]:
database = pickle.load( open( "Mod_AnyT_DB.p", "rb" ) )

In [274]:
for i in range(len(database)):
    diams = dict(zip(wn_current_WDS.link_name_list, list(database.loc[i]['Diams']))) #diam
    G_WDS = wn_current_WDS.get_graph(link_weight=diams) # directed multigraph
    #A_WDS = nx.adjacency_matrix(G_WDS)
    uG_WDS = G_WDS.to_undirected()
    A_WDS = nx.normalized_laplacian_matrix(uG_WDS)
    #A_WDS -=  identity(21)
    S_matrices.append(A_WDS.todense())
    
    res_index.append(database.loc[i]['avgPrPa'])
    

In [276]:
S_matrices = torch.tensor(np.array(S_matrices), dtype = torch.float32)
res_index = torch.tensor(np.array(res_index), dtype = torch.float32)

In [277]:
demands=[]
for i in wn_current_WDS.node_name_list:
    a = wn_current_WDS.get_node(i)
    try:
        demands.append(a.base_demand)
    except Exception as e:
        #demands.append(-a.base_head)  
        demands.append(0)
        print(e)
        
        

'Reservoir' object has no attribute 'base_demand'
'Reservoir' object has no attribute 'base_demand'


In [291]:
class GNN_2(nn.Module):
    def __init__(self, units, k=1):  #second_filter = False
        super(GNN_2, self).__init__()
        self.filters = nn.ModuleList()
        for i in range(len(units)-1):
            self.filters.append(nn.ParameterList([nn.Parameter(torch.randn(units[i],units[i+1])) for j in range(k)]))
            
            
        self.activations = nn.ModuleDict([
                ['relu', nn.ReLU()],
                ['tanh', nn.Tanh()]
        ])

    def forward(self, S, x):
        #Implement matrix power once at the beginning
        
        emb = [x]
        filt = 0
        
        for list_h_k in self.filters:
            k=0
            ans_f = torch.empty(S.shape[-1], list_h_k[0].shape[-1]) #Nodes x hid
            for h_k in list_h_k:
                Sx = torch.spmm(torch.matrix_power(S,k), emb[filt])
                SxH = torch.mm(Sx, h_k)
                k+=1
                ans_f += SxH
            print(Sx)
            emb.append(self.activations['relu'](ans_f))
            filt +=1
       
        ans = self.activations['tanh'](torch.sum(emb[-1]))
        #ans = emb
        return ans
    
    # Calculates the number of parameters for each layer
    def num_params(self):
        self.num_params = []
        total = 0
        for i in self.parameters():
            current_number = torch.numel(i)
            self.num_params.append(current_number)
            total+=current_number
        
        return self.num_params
        

In [292]:
model = GNN_2(units = [1, 5, 1])

C:\Programs\Anaconda3\envs\PyTorch_Env\lib\site-packages\torch\nn\modules\container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [290]:
model(S_matrices[10], x)

tensor(0.3541, grad_fn=<TanhBackward>)

In [278]:
S = torch.tensor(np.array(A_WDS.todense()), dtype = torch.float32)
x = torch.tensor(np.array(demands), dtype = torch.float32)
x = (x - torch.min(x))/(torch.max(x)-torch.min(x))
x = x.reshape(21, 1)

In [279]:
class GNN(nn.Module):
    def __init__(self, x_feats, hid_feats, out_feats, second_filter = False):  #second_filter = False
        super(GNN, self).__init__()
        
        self.second_filter = second_filter
        
        self.h_1_0 = nn.Parameter(torch.rand(x_feats,hid_feats))
        self.h_1_1 = nn.Parameter(torch.rand(x_feats,hid_feats))
        self.h_1_2 = nn.Parameter(torch.rand(x_feats,hid_feats))
        
        #if second_filter:
        self.h_2_0 = nn.Parameter(torch.rand(hid_feats, out_feats))
        self.h_2_1 = nn.Parameter(torch.rand(hid_feats, out_feats))
        self.h_2_2 = nn.Parameter(torch.rand(hid_feats, out_feats))
        
#         self.h_2_0 = nn.Parameter(torch.rand(1, 5))
#         self.h_2_1 = nn.Parameter(torch.rand(1, 5))
#         self.h_2_2 = nn.Parameter(torch.rand(1, 5))
        
        #self.fc = nn.Linear(21, 1)
        
    #def graph_filter(self, S, x, ):
        
    def forward(self, S, x):
        
        f1 =  torch.matmul(torch.matmul(torch.pow(S, 0), x),self.h_1_0)
        f1 += torch.matmul(torch.matmul(torch.pow(S, 1), x),self.h_1_1)
        f1 += torch.matmul(torch.matmul(torch.pow(S, 2), x), self.h_1_2)
        
        f1 = torch.sigmoid(f1)
        
        #if self.second_filter:
#         f2 =  torch.matmul(f1, torch.matmul(torch.pow(S, 0), self.h_2_0))
#         f2 += torch.matmul(f1, torch.matmul(torch.pow(S, 1), self.h_2_1))
#         f2 += torch.matmul(f1, torch.matmul(torch.pow(S, 2), self.h_2_2))
        
#         f2 = torch.sigmoid(f2)
        
        
        ans = torch.mean(f1)
        ans = torch.tanh(ans)
        
        #ans = torch.sigmoid(ans)
        
        #ans = torch.relu(ans)
        
        return ans
    def num_params(self):
        self.num_params = []
        total = 0
        for i in self.parameters():
            current_number = torch.numel(i)
            self.num_params.append(current_number)
            total+=current_number
        
        return self.num_params


In [280]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GNN_2(units = [1, 100, 50, 1]).to(device)
model = GNN(1, 100, 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)


In [282]:
training_range = 2000
test_range = 1000

In [ ]:
for epoch in range(91):
    total_loss= 0
#     total_correct =0
    #Data
    for i in range(training_range):
        #Prediction
        preds = model(S_matrices[i], x)
        labels = res_index[i]

        #Calculate the loss
        loss = F.mse_loss(preds.reshape(-1,1), labels.reshape(-1,1)) 

        #Backpropagate
        optimizer.zero_grad() #To avoid adding up gradients
        loss.backward() #calculate gradients

        #Optimizer step
        optimizer.step() #Update weights

        total_loss += loss.item()
#         total_correct += get_number_correct_labels(preds, labels)
    if epoch%10 ==0:
        print("epoch ", epoch, " total loss  ", total_loss )

In [284]:
model.num_params()

[100, 100, 100, 100, 100, 100]

In [ ]:
for i in model.parameters():
    print(i)

In [285]:
preds

tensor(0.3541, grad_fn=<TanhBackward>)

In [286]:
model(S_matrices[95],x)

tensor(0.3541, grad_fn=<TanhBackward>)

In [287]:
res_index[95]

tensor(0.4141)

In [288]:
total_loss/training_range

0.0024895664328980524

In [289]:
loss_dev = 0
for i in range(training_range,training_range+test_range):
    #Prediction
    preds = model(S_matrices[i], x)
    labels = res_index[i]
    print(np.round(preds.item(), 4), labels)
    #Calculate the loss
    loss_dev += F.mse_loss(preds.reshape(1), labels.reshape(1))

print(loss_dev/test_range)

0.3541 tensor(0.4008)
0.3541 tensor(0.1595)
0.3541 tensor(0.3524)
0.3541 tensor(0.4565)
0.3541 tensor(0.4069)
0.3541 tensor(0.3403)
0.3541 tensor(0.3723)
0.3541 tensor(0.1786)
0.3541 tensor(0.3115)
0.3541 tensor(0.4135)
0.3541 tensor(0.3659)
0.3541 tensor(0.2052)
0.3541 tensor(0.3923)
0.3541 tensor(0.3494)
0.3541 tensor(0.3278)
0.3541 tensor(0.4150)
0.3541 tensor(0.4104)
0.3541 tensor(0.3042)
0.3541 tensor(0.4668)
0.3541 tensor(0.3735)
0.3541 tensor(0.2731)
0.3541 tensor(0.2628)
0.3541 tensor(0.3442)
0.3541 tensor(0.3981)
0.3541 tensor(-0.1533)
0.3541 tensor(-0.1327)
0.3541 tensor(0.3802)
0.3541 tensor(0.4245)
0.3541 tensor(0.4533)
0.3541 tensor(0.4602)
0.3541 tensor(0.4536)
0.3541 tensor(0.3795)
0.3541 tensor(0.3024)
0.3541 tensor(0.3857)
0.3541 tensor(0.4579)
0.3541 tensor(0.3214)
0.3541 tensor(0.2902)
0.3541 tensor(0.4900)
0.3541 tensor(0.3988)
0.3541 tensor(0.3914)
0.3541 tensor(0.3195)
0.3541 tensor(0.4471)
0.3541 tensor(0.1125)
0.3541 tensor(0.3990)
0.3541 tensor(0.3093)
0.3541 t

In [ ]:
torch.sum(res_index)/5000